In [19]:
import pandas as pd

In [20]:
chr = pd.read_csv('../00_data/02_intermediate/chr.csv')
gdp = pd.read_csv('../00_data/02_intermediate/gdp.csv')
noise = pd.read_csv('../00_data/02_intermediate/noise.csv')
place = pd.read_csv('../00_data/02_intermediate/place.csv')

res = pd.read_excel('../00_data/01_raw/2025-food-environment-atlas-data.xlsx', sheet_name="RESTAURANTS",skiprows=1)


In [21]:
chr.columns

Index(['adult_smoking', 'adult_obesity', 'sti', 'excessive_drinking',
       'population', 'under_18_pct', 'diabetes_prevalence', 'no_time_activity',
       'some_college', 'unemployment', 'children_single_parent',
       'mental_health_providers', 'median_household_income',
       'driving_alone_to_work', 'life_expectancy', 'fipscode'],
      dtype='object')

In [22]:

chr_renamed = chr[['fipscode',
       'adult_smoking', 'adult_obesity', 'excessive_drinking',
       'diabetes_prevalence', 'some_college','no_time_activity',
       'unemployment', 'children_single_parent', 'mental_health_providers',
       'median_household_income', 'driving_alone_to_work', 'sti','life_expectancy']].rename(columns={'fipscode': 'FIPS'})

place_renamed = place.rename(columns={'LocationID': 'FIPS'})

gdp2022 = gdp.loc[gdp['year']==2022]
gdp_renamed = gdp2022.rename(columns={'GeoFIPS': 'FIPS'})


noise['geoid_short'] = noise['geoid_short'].astype(str)

def strip_zeros(x):
    return x.lstrip("0").rstrip("0")

noise['FIPS'] = noise['geoid_short'].apply(strip_zeros)

noise_select = noise[['coverage_50','coverage_60', 'coverage_70', 'coverage_80', 'coverage_90', 
       'noise50n', 'noise60n','noise70n', 'noise80n', 'noise90n', 'FIPS']]

In [23]:
df1 = pd.merge(place_renamed,chr_renamed, on='FIPS', how='left')
df2 = pd.merge(df1, res[['FIPS','FFR20']], on='FIPS', how='left')
df3 = pd.merge(df2, gdp_renamed[['FIPS','gdp']], on='FIPS', how='left')
df3['FIPS'] = df3['FIPS'].astype(str)
final = pd.merge(df3, noise_select, on='FIPS', how='left')


In [24]:
final

,StateAbbr,LocationName,FIPS,TotalPopulation,BINGE,CANCER,CSMOKING,DEPRESSION,DIABETES,DISABILITY,...,coverage_50,coverage_60,coverage_70,coverage_80,coverage_90,noise50n,noise60n,noise70n,noise80n,noise90n
0,AK,Aleutians East,2013,3398,560.670,139.318,618.436,421.352,434.944,1002.410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,Aleutians West,2016,5122,1044.888,204.880,896.350,691.470,548.054,1347.086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,Anchorage,2020,287145,55993.275,18664.425,38477.430,60874.740,22971.600,74083.410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AK,Bethel,2050,18257,3468.830,967.621,6079.581,4107.825,2428.181,7978.309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,Bristol Bay,2060,865,176.460,70.065,150.510,173.000,94.285,269.015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,WY,Sweetwater,56037,41345,7938.240,3224.910,7318.065,9013.210,4217.190,12858.295,...,166680.258452,49899.968826,15487.437393,1218.442079,140.0,5383.728053,1116.699341,328.074876,40.555948,4.603305
3140,WY,Teton,56039,23287,4820.409,1932.821,2654.718,4447.817,1839.673,5286.149,...,31196.540103,6304.316617,369.000000,18.000000,0.0,804.428939,141.155460,10.876001,0.323381,0.000000
3141,WY,Uinta,56041,20712,3914.568,1719.096,3686.736,4846.608,2154.048,6544.992,...,61544.797830,16958.235114,5934.810009,387.960956,58.0,1531.873214,385.804989,123.159489,5.249862,0.410199
3142,WY,Washakie,56043,7719,1211.883,764.181,1335.387,1590.114,949.437,2593.584,...,11212.993376,2599.774364,132.000000,33.000000,7.0,349.434177,54.242065,5.574692,1.160815,0.873946


In [25]:
final.to_csv('../00_data/03_final/county_level.csv',index=False)

---

In [26]:
df = pd.read_csv('https://data.nber.org/cbsa-csa-fips-county-crosswalk/cbsa2fipsxw.csv')
park = pd.read_csv('../00_data/02_intermediate/cbsa_park.csv')


In [27]:
df["fipsstatecode"] = df["fipsstatecode"].astype(str)
df["fipscountycode"] = df["fipscountycode"].astype(str)

df["fipsstatecode"] = df["fipsstatecode"].str.zfill(2) 
df["fipscountycode"] = df["fipscountycode"].str.zfill(3)

df["countyfips"] = df["fipsstatecode"] + df["fipscountycode"]
df["FIPS"] = df["countyfips"].apply(strip_zeros)

park["cbsacode"] = park["CBSA Code"]

In [28]:
df4 = pd.merge(final,df[["FIPS", "cbsacode"]], on='FIPS', how='left')

In [29]:
cols = ['TotalPopulation','BINGE','CSMOKING','DEPRESSION','DIABETES','LPA','OBESITY',
        'adult_smoking','adult_obesity','excessive_drinking','diabetes_prevalence',
        'some_college','unemployment','children_single_parent','mental_health_providers',
        'median_household_income','driving_alone_to_work','sti','FFR20','gdp',
        'coverage_50','coverage_60','coverage_70','coverage_80','coverage_90',
        'noise50n','noise60n','noise70n','noise80n','noise90n','CANCER','VISION',
       'MOBILITY','life_expectancy',
       'SELFCARE',
       'DISABILITY']

tmp = df4.dropna(subset=['cbsacode']).copy()

# ensure numeric (non-numeric → NaN)
tmp[cols] = tmp[cols].apply(pd.to_numeric, errors='coerce')

# sum within CBSA (min_count=1 avoids all-NaN → 0)
clean = (tmp
         .groupby('cbsacode')[cols]
         .sum(min_count=1)
         .reset_index()).dropna()

df5 = pd.merge(clean,park[["cbsacode", "Park_Area_Acres"]], on='cbsacode', how='left').dropna()

In [30]:
df5['cbsacode'].unique().shape

(875,)

In [31]:
road = pd.read_csv('../00_data/02_intermediate/road_cbsa.csv')
road.rename(columns={'CBSA':'cbsacode'}, inplace=True)

In [32]:
road

,cbsacode,road_network_total,auto_net_total,multimodal_net_total,pedestrian_net_total,street_intersection_total,auto_intersections_total,multimodal_3leg_intersections_total,multimodal_4leg_intersections_total,pedestrian_3leg_intersections_total,pedestrian_4leg_intersections_total,Ac_Land
0,10100.0,3.284660e+06,405321.770783,2.646972e+06,2.323662e+05,2.253482e+06,4.416000e+05,8.025600e+05,824960.000122,6.022400e+05,4.915200e+05,1.816927e+06
1,10140.0,2.374547e+06,104440.891567,1.599555e+05,2.110150e+06,2.589727e+06,1.440000e+05,4.236800e+05,113919.999991,2.356480e+06,6.214400e+05,1.216910e+06
2,10180.0,4.222981e+06,649977.555417,1.960052e+06,1.612951e+06,6.348198e+06,1.219200e+06,2.706560e+06,566400.000211,3.808640e+06,1.436160e+06,1.755788e+06
3,10220.0,1.003198e+06,118841.502159,5.561159e+05,3.282406e+05,1.655392e+06,2.086400e+05,8.627200e+05,156159.999958,8.332800e+05,3.680000e+05,4.610480e+05
4,10300.0,1.318877e+06,241518.103246,8.121703e+05,2.651885e+05,1.677814e+06,4.224000e+05,8.582400e+05,213120.000005,9.030400e+05,2.899200e+05,4.797457e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
933,49660.0,4.360935e+06,806578.554084,1.665215e+06,1.889141e+06,9.193127e+06,8.960000e+05,3.553280e+06,935679.999969,5.907840e+06,1.946880e+06,1.089320e+06
934,49700.0,2.180572e+06,336653.884516,1.065351e+05,1.737383e+06,3.295718e+06,5.132800e+05,3.641600e+05,113280.000069,3.271040e+06,7.577600e+05,7.900925e+05
935,49740.0,2.408522e+06,217941.354627,1.111215e+06,1.079365e+06,4.359259e+06,1.484800e+05,1.288960e+06,476159.999328,3.207680e+06,8.838400e+05,3.528780e+06
936,49780.0,1.206591e+06,188083.838884,7.821741e+05,2.363330e+05,1.744664e+06,2.924800e+05,1.165440e+06,162560.000015,8.256000e+05,2.540800e+05,4.253001e+05


In [33]:
df6 = pd.merge(df5, road, on='cbsacode', how='left')

In [34]:
df6.columns

Index(['cbsacode', 'TotalPopulation', 'BINGE', 'CSMOKING', 'DEPRESSION',
       'DIABETES', 'LPA', 'OBESITY', 'adult_smoking', 'adult_obesity',
       'excessive_drinking', 'diabetes_prevalence', 'some_college',
       'unemployment', 'children_single_parent', 'mental_health_providers',
       'median_household_income', 'driving_alone_to_work', 'sti', 'FFR20',
       'gdp', 'coverage_50', 'coverage_60', 'coverage_70', 'coverage_80',
       'coverage_90', 'noise50n', 'noise60n', 'noise70n', 'noise80n',
       'noise90n', 'CANCER', 'VISION', 'MOBILITY', 'life_expectancy',
       'SELFCARE', 'DISABILITY', 'Park_Area_Acres', 'road_network_total',
       'auto_net_total', 'multimodal_net_total', 'pedestrian_net_total',
       'street_intersection_total', 'auto_intersections_total',
       'multimodal_3leg_intersections_total',
       'multimodal_4leg_intersections_total',
       'pedestrian_3leg_intersections_total',
       'pedestrian_4leg_intersections_total', 'Ac_Land'],
      dtype='obj

In [35]:
df6['cbsacode'].unique().shape

(875,)

In [36]:
df6.to_csv('../00_data/03_final/cbsa_level.csv',index=False)